# **Modelagem Estatística**
## Data Extraction
Felipe Lamarca

Vamos extrair os dados da **Base dos Dados** utilizando queries SQL. Para isso, vamos utilizar o pacote `basedosdados`, disponível em Python e R.

Os dados são extraídos das tabelas `candidatos`, `bens_candidato` e `despesas_candidato` da base de dados `br_tse_eleicoes`.
Acesse através do [link](https://basedosdados.org/dataset/eef764df-bde8-4905-b115-6fc23b6ba9d6?table=2e204854-e453-4257-9fef-5e10f3ff1f56).

- A tabela `candidatos` contém informações sobre os candidatos, como nome, partido, cargo, gênero, raça, etc.
- A tabela `bens_candidato` contém informações sobre os bens declarados pelos candidatos.
- A tabela `despesas_candidato` contém informações sobre as despesas de campanha dos candidatos.

Vamos extrair os dados dos anos de 2018 e 2022, para todos os estados brasileiros, para o cargo de deputado federal.
Queremos avaliar os dados apenas para o ano de 2022, mas acessamos os dados de 2018 para verificar se o candidato está
tentando a reeleição.

In [9]:
import basedosdados as bd
import pandas as pd

In [10]:
billing_id = input("Digite o seu billing id: ")
# "felipe-388411"

In [11]:
# WRITE THE QUERIES
query1 = """SELECT DISTINCT c.ano, c.sigla_uf, c.id_candidato_bd, c.nome_urna, c.sigla_partido, c.cargo,
c.genero, c.raca, rc.resultado, SUM(rc.votos) votos
FROM basedosdados.br_tse_eleicoes.candidatos c
INNER JOIN basedosdados.br_tse_eleicoes.resultados_candidato rc
ON c.id_candidato_bd = rc.id_candidato_bd
AND c.ano = rc.ano
AND c.cargo = 'deputado federal'
AND c.ano in (2022, 2018)
GROUP BY c.ano, c.sigla_uf, c.id_candidato_bd, c.nome_urna, c.sigla_partido, c.cargo,
c.genero, c.raca, rc.resultado"""

query2 = """SELECT DISTINCT c.ano, c.id_candidato_bd, SUM(bc.valor_item) bens_declarados
FROM basedosdados.br_tse_eleicoes.candidatos c
INNER JOIN basedosdados.br_tse_eleicoes.bens_candidato bc
ON c.id_candidato_bd = bc.id_candidato_bd
AND c.ano = bc.ano
AND c.cargo = 'deputado federal'
AND c.ano in (2022, 2018)
GROUP BY c.ano, c.id_candidato_bd"""

query3 = """SELECT DISTINCT c.ano, c.id_candidato_bd, SUM(dc.valor_despesa) despesas_campanha
FROM basedosdados.br_tse_eleicoes.candidatos c
INNER JOIN basedosdados.br_tse_eleicoes.despesas_candidato dc
ON c.id_candidato_bd = dc.id_candidato_bd
AND c.ano = dc.ano
AND c.cargo = 'deputado federal'
AND c.ano in (2022, 2018)
GROUP BY c.ano, c.id_candidato_bd"""

candidatos = bd.read_sql(query1, billing_project_id=billing_id)
bens = bd.read_sql(query2, billing_project_id=billing_id)
despesas = bd.read_sql(query3, billing_project_id=billing_id)

Downloading: 100%|██████████| 14201/14201 [00:01<00:00, 14063.94rows/s]


Após acessar os dados, vamos juntá-los em um único arquivo `.csv`.

In [12]:
# join the data
df = pd.merge(candidatos, bens, on=['ano', 'id_candidato_bd'], how='left')
df = pd.merge(df, despesas, on=['ano', 'id_candidato_bd'], how='left')

# save the data
df.to_csv('../data/data1822.csv', index=False)